# Weather Bot com Python Notebook

Este notebook demonstra como criar um agente de clima inteligente usando o Google AI Development Kit (ADK). O agente será capaz de:
- Responder consultas sobre o clima
- Delegar tarefas para sub-agentes especializados
- Manter estado e memória entre conversas
- Aplicar guardrails de segurança

In [ ]:
# @title Etapa 0: Configuração e Instalação de Dependências
"""
Esta célula instala as bibliotecas necessárias para o projeto:
- google-adk: Kit de desenvolvimento de agentes do Google
- litellm: Biblioteca para integração com múltiplos modelos de LLM
- python-dotenv: Para carregar variáveis de ambiente de arquivos .env
"""

# Instala o ADK e LiteLLM para suporte multi-modelo
!pip install google-adk -q
!pip install litellm -q
!pip install python-dotenv

print("Instalação concluída com sucesso.")

## Etapa 1: Configuração - Pesquisa básica do clima

Nesta primeira etapa, vamos configurar um agente básico de clima que pode:
- Importar as bibliotecas necessárias
- Configurar chaves de API
- Definir ferramentas para buscar informações meteorológicas
- Criar um agente simples que responde a consultas sobre o clima

In [ ]:
# @title Importação das Bibliotecas Necessárias
"""
Esta célula importa todas as bibliotecas essenciais para criar agentes de IA:
- os, asyncio: Bibliotecas padrão do Python para sistema e programação assíncrona
- google.adk: Componentes principais do Google AI Development Kit
- google.genai: Tipos para criação de conteúdo de mensagens
- warnings, logging: Para controle de avisos e logs
"""

import os
import asyncio
from google.adk.agents import Agent  # Classe principal para criar agentes
from google.adk.sessions import InMemorySessionService  # Gerenciamento de sessões em memória
from google.adk.runners import Runner  # Orquestrador de execução de agentes
from google.genai import types  # Para criar conteúdo/partes de mensagens
#from google.adk.models.lite_llm import LiteLlm  # Comentado para evitar problemas de compatibilidade

import warnings
warnings.filterwarnings("ignore")  # Suprime avisos desnecessários

import logging
logging.basicConfig(level=logging.ERROR)  # Configura logging apenas para erros

print("✅ Bibliotecas importadas com sucesso.")

In [ ]:
# @title Configuração das Chaves de API
"""
Esta célula configura as chaves de API necessárias para usar os modelos de IA:
- Carrega variáveis de ambiente do arquivo .env
- Configura chave do Google AI (obrigatória)
- Opcionalmente configura OpenAI e Anthropic (comentado)
- Verifica se as chaves foram definidas corretamente
"""

import os
from dotenv import load_dotenv

# Carrega variáveis de ambiente do arquivo .env
load_dotenv() 

# IMPORTANTE: Descomente e adicione sua chave real do Google AI
#os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_AQUI"

'''# [Opcional] Chaves para outros provedores de LLM
os.environ['OPENAI_API_KEY'] = 'SUA_CHAVE_OPENAI_AQUI'
os.environ['ANTHROPIC_API_KEY'] = 'SUA_CHAVE_ANTHROPIC_AQUI'
'''

# --- Verificação das Chaves (Verificação Opcional) ---
print("🔑 Status das Chaves de API:")
print(f"Google API Key: {'✅ Configurada' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else '❌ NÃO CONFIGURADA (SUBSTITUA O PLACEHOLDER!)'}")

'''# Verificações para outros provedores (comentado)
print(f"OpenAI API Key: {'✅ Configurada' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else '❌ Não configurada'}")
print(f"Anthropic API Key: {'✅ Configurada' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else '❌ Não configurada'}")
'''

# Configura o ADK para usar chaves de API diretamente (não Vertex AI)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

In [ ]:
# @title Definição de Constantes de Modelos
"""
Esta célula define constantes para facilitar o uso de diferentes modelos de IA:
- MODEL_GEMINI_2_0_FLASH: Modelo principal do Google (gratuito com limites)
- Outros modelos comentados para referência futura
"""

# --- Definir Constantes de Modelo para facilitar o uso ---

# Modelos do Google Gemini - Documentação: https://ai.google.dev/gemini-api/docs/models#model-variations
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"  # Modelo principal que usaremos

'''# Modelos OpenAI via LiteLLM (comentado)
# Documentação: https://docs.litellm.ai/docs/providers/openai#openai-chat-completion-models
MODEL_GPT_4O = "openai/gpt-4o"  # Também disponível: gpt-4o-mini, gpt-4-turbo, etc.

# Modelos Anthropic via LiteLLM (comentado)  
# Documentação: https://docs.litellm.ai/docs/providers/anthropic
MODEL_CLAUDE_SONNET = "anthropic/claude-3-5-sonnet-20241022"  # Também: claude-3-opus, claude-3-haiku
'''

print("🤖 Ambiente configurado e modelos definidos.")

### 🛠️ Definindo Ferramentas (Tools)

**Conceitos importantes:**
- **Uma ferramenta**: Uma função Python que equipa o agente com habilidades específicas para buscar dados meteorológicos
- **Um agente**: O "cérebro" da IA que entende solicitações do usuário, sabe quais ferramentas possui e decide quando/como usá-las

As ferramentas são o meio pelo qual os agentes interagem com o mundo externo.

In [ ]:
# @title Definindo a Ferramenta get_weather
"""
Esta função simula uma API de clima e será usada como ferramenta pelo agente.
Características:
- Recebe o nome de uma cidade como parâmetro
- Retorna informações meteorológicas mockadas (simuladas)
- Suporta algumas cidades predefinidas (New York, London, Tokyo)
- Retorna erro para cidades não suportadas
"""

def get_weather(city: str) -> dict:
    """Recupera informações meteorológicas para uma cidade específica.

    Args:
        city (str): Nome da cidade (ex: "New York", "London", "Tokyo").

    Returns:
        dict: Dicionário contendo informações meteorológicas.
              Inclui uma chave 'status' ('success' ou 'error').
              Se 'success', inclui chave 'report' com detalhes do clima.
              Se 'error', inclui chave 'error_message'.
    """
    print(f"🌤️ Ferramenta: get_weather chamada para cidade: {city}")
    
    # Normalização básica do nome da cidade
    city_normalized = city.lower().replace(" ", "")

    # Base de dados mockada com informações meteorológicas
    mock_weather_db = {
        "newyork": {
            "status": "success", 
            "report": "O clima em Nova York está ensolarado com temperatura de 25°C."
        },
        "london": {
            "status": "success", 
            "report": "Está nublado em Londres com temperatura de 15°C."
        },
        "tokyo": {
            "status": "success", 
            "report": "Tóquio está com chuva leve e temperatura de 18°C."
        },
    }

    # Verifica se a cidade está na base de dados
    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {
            "status": "error", 
            "error_message": f"Desculpe, não tenho informações meteorológicas para '{city}'."
        }

# Exemplos de uso da ferramenta (teste opcional)
print("🧪 Testando a ferramenta:")
print(get_weather("New York"))
print(get_weather("Paris"))  # Cidade não suportada

### 🤖 Definindo o Agente Principal

Agora vamos criar o Agente propriamente dito. Um **Agent** no ADK orquestra a interação entre:
- O usuário (que faz perguntas)
- O LLM (que processa e entende as solicitações)  
- As ferramentas disponíveis (que executam ações específicas)

O agente decide quando e como usar cada ferramenta baseado nas instruções que fornecemos.

In [ ]:
# @title Criando o Agente de Clima
"""
Esta célula cria o primeiro agente de clima com as seguintes características:
- Nome: weather_agent_v1
- Modelo: Gemini 2.0 Flash
- Ferramenta: get_weather (definida anteriormente)
- Instruções claras sobre como e quando usar a ferramenta
"""

# Usa uma das constantes de modelo definidas anteriormente
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH

# Criação do agente principal
weather_agent = Agent(
    name="weather_agent_v1",  # Nome identificador do agente
    model=AGENT_MODEL,  # Pode ser string para Gemini ou objeto LiteLlm
    description="Fornece informações meteorológicas para cidades específicas.",
    
    # Instruções detalhadas que definem o comportamento do agente
    instruction="Você é um assistente meteorológico útil. "
                "Quando o usuário perguntar sobre o clima de uma cidade específica, "
                "use a ferramenta 'get_weather' para encontrar as informações. "
                "Se a ferramenta retornar um erro, informe o usuário educadamente. "
                "Se a ferramenta for bem-sucedida, apresente o relatório meteorológico claramente.",
    
    tools=[get_weather],  # Lista de ferramentas disponíveis para este agente
)

print(f"✅ Agente '{weather_agent.name}' criado usando modelo '{AGENT_MODEL}'.")

### ⚙️ Configurar Runner e Serviço de Sessão

Antes de poder usar o agente, precisamos configurar:
- **SessionService**: Gerencia o histórico de conversas e estado entre interações
- **Runner**: Orquestra a execução do agente e gerencia o fluxo de eventos

Estes componentes trabalham juntos para criar uma experiência de conversa fluida.

In [ ]:
# @title Configuração do Serviço de Sessão e Runner
"""
Esta célula configura os componentes necessários para executar o agente:
1. SessionService: Armazena histórico de conversas e estado
2. Constantes de identificação: App, usuário e sessão
3. Runner: Orquestrador que executa o agente
"""

# --- Gerenciamento de Sessão ---
# Conceito-chave: SessionService armazena histórico de conversas e estado.
# InMemorySessionService é um armazenamento simples e não-persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar o contexto da interação
APP_NAME = "weather_tutorial_app"  # Nome da aplicação
USER_ID = "user_1"                 # ID do usuário
SESSION_ID = "session_001"         # ID da sessão (fixo para simplicidade)

# Criar a sessão específica onde a conversa acontecerá
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"📝 Sessão criada: App='{APP_NAME}', Usuário='{USER_ID}', Sessão='{SESSION_ID}'")

# --- Runner ---
# Conceito-chave: Runner orquestra o loop de execução do agente.
runner = Runner(
    agent=weather_agent,              # O agente que queremos executar
    app_name=APP_NAME,               # Associa execuções com nossa aplicação
    session_service=session_service  # Usa nosso gerenciador de sessão
)
print(f"🚀 Runner criado para agente '{runner.agent.name}'.")

### 💬 Interação com o Agente

Agora vamos criar funções para interagir com nosso agente. O processo envolve:
1. Enviar uma mensagem do usuário
2. O agente processa e decide se precisa usar ferramentas
3. Receber e exibir a resposta final

In [ ]:
# @title Função de Interação com o Agente (Versão Básica)
"""
Esta função permite enviar mensagens para o agente e receber respostas.
Características desta versão básica:
- Envia uma consulta de texto para o agente
- Processa eventos de execução em tempo real
- Extrai e exibe a resposta final
"""

from google.genai import types  # Para criar conteúdo/partes de mensagens

async def call_agent_async(query: str, runner, user_id, session_id):
    """Envia uma consulta para o agente e exibe a resposta final."""
    print(f"\n💭 Consulta do Usuário: {query}")

    # Preparar a mensagem do usuário no formato ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agente não produziu uma resposta final."  # Padrão

    # Conceito-chave: run_async executa a lógica do agente e gera Eventos.
    # Iteramos através dos eventos para encontrar a resposta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Descomente a linha abaixo para ver *todos* os eventos durante a execução
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Conteúdo: {event.content}")

        # Conceito-chave: is_final_response() marca a mensagem conclusiva do turno.
        if event.is_final_response():
            if event.content and event.content.parts:
               # Assumindo resposta de texto na primeira parte
               final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:  # Lidar com possíveis erros/escalações
               final_response_text = f"Agente escalou: {event.error_message or 'Nenhuma mensagem específica.'}"
            # Adicionar mais verificações aqui se necessário (ex: códigos de erro específicos)
            break  # Parar processamento de eventos após encontrar a resposta final

    print(f"🤖 Resposta do Agente: {final_response_text}")

### 🎬 Execução da Conversa

Agora vamos testar nosso agente com algumas consultas meteorológicas. Teste casos incluem:
- Cidade suportada (deve usar a ferramenta)
- Cidade não suportada (deve retornar erro educadamente)
- Múltiplas consultas na mesma sessão

In [ ]:
# @title [CÉLULA DUPLICADA - PODE SER REMOVIDA]
"""
Esta célula parece ser uma duplicação da função call_agent_async definida anteriormente.
Pode ser removida com segurança.
"""

# Esta função já foi definida na célula anterior
# Mantida apenas para compatibilidade, mas pode ser removida

In [ ]:
# @title Executando a Conversa Inicial
"""
Esta célula testa o agente com diferentes tipos de consultas:
1. Cidade suportada (Londres) - deve funcionar
2. Cidade não suportada (Paris) - deve retornar erro educado
3. Outra cidade suportada (Nova York) - deve funcionar
"""

# Precisamos de uma função async para aguardar nosso helper de interação
async def run_conversation():
    """Executa uma conversa de teste com o agente meteorológico."""
    
    # Teste 1: Cidade suportada
    await call_agent_async("Como está o clima em Londres?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    # Teste 2: Cidade não suportada (esperando mensagem de erro da ferramenta)
    await call_agent_async("E em Paris?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    # Teste 3: Outra cidade suportada
    await call_agent_async("Me fale sobre o clima em Nova York",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

# Executar a conversa usando await em contexto async (como Colab/Jupyter)
print("🎬 Iniciando conversa de teste...")
await run_conversation()

# --- ALTERNATIVA ---
# Descomente as linhas seguintes se executando como script Python padrão (.py):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"Ocorreu um erro: {e}")

## Etapa 3: Construção de uma Equipe de Agentes

Nesta etapa, vamos evoluir nosso sistema criando **múltiplos agentes especializados**:

- **Agente de Saudação**: Especializado em cumprimentos
- **Agente de Despedida**: Especializado em despedidas  
- **Agente Principal**: Coordena e delega tarefas para os especialistas

Isso demonstra como criar uma arquitetura de agentes colaborativa onde cada um tem responsabilidades específicas.

In [ ]:
# @title Definindo Ferramentas para Agentes de Saudação e Despedida
"""
Esta célula cria ferramentas especializadas para interações sociais:
- say_hello: Gera saudações personalizadas (com ou sem nome)
- say_goodbye: Gera despedidas padronizadas

Estas ferramentas serão usadas por agentes especializados.
"""

from typing import Optional  # Certifica-se de importar Optional

# Garante que 'get_weather' da Etapa 1 esteja disponível se executando esta etapa independentemente
# def get_weather(city: str) -> dict: ... (da Etapa 1)

def say_hello(name: Optional[str] = None) -> str:
    """Fornece uma saudação simples. Se um nome for fornecido, será usado.

    Args:
        name (str, optional): Nome da pessoa a ser cumprimentada. 
                             Padrão para saudação genérica se não fornecido.

    Returns:
        str: Mensagem de saudação amigável.
    """
    if name:
        greeting = f"Olá, {name}!"
        print(f"👋 Ferramenta: say_hello chamada com nome: {name}")
    else:
        greeting = "Olá!"  # Saudação padrão se name é None ou não foi explicitamente passado
        print(f"👋 Ferramenta: say_hello chamada sem nome específico (valor_arg_nome: {name})")
    return greeting

def say_goodbye() -> str:
    """Fornece uma mensagem de despedida simples para concluir a conversa."""
    print(f"👋 Ferramenta: say_goodbye chamada")
    return "Tchau! Tenha um ótimo dia."

print("✅ Ferramentas de saudação e despedida definidas.")

# Autoteste opcional
print("\n🧪 Testando as ferramentas:")
print(say_hello("Alice"))
print(say_hello())  # Teste sem argumento (deve usar padrão "Olá!")
print(say_hello(name=None))  # Teste com name explicitamente como None (deve usar padrão)
print(say_goodbye())

In [ ]:
# @title Criando Sub-Agentes Especializados
"""
Esta célula cria dois agentes especializados:
1. Greeting Agent: Focado apenas em saudações usando say_hello
2. Farewell Agent: Focado apenas em despedidas usando say_goodbye

Cada agente tem instruções muito específicas para sua tarefa.
"""

# Se quiser usar modelos diferentes do Gemini, certifique-se de que LiteLlm esteja importado e chaves de API definidas
# from google.adk.models.lite_llm import LiteLlm
# MODEL_GPT_4O, MODEL_CLAUDE_SONNET etc. devem estar definidos
# Caso contrário, continue usando: model = MODEL_GEMINI_2_0_FLASH

# --- Agente de Saudação ---
greeting_agent = None
try:
    greeting_agent = Agent(
        # Usando um modelo potencialmente diferente/mais barato para uma tarefa simples
        model=MODEL_GEMINI_2_0_FLASH,
        # model=LiteLlm(model=MODEL_GPT_4O),  # Se quiser experimentar com outros modelos
        name="greeting_agent",
        instruction="Você é o Agente de Saudação. Sua ÚNICA tarefa é fornecer uma saudação amigável ao usuário. "
                    "Use a ferramenta 'say_hello' para gerar a saudação. "
                    "Se o usuário fornecer seu nome, certifique-se de passá-lo para a ferramenta. "
                    "Não se envolva em outras conversas ou tarefas.",
        description="Lida com saudações simples usando a ferramenta 'say_hello'.",  # Crucial para delegação
        tools=[say_hello],
    )
    print(f"✅ Agente '{greeting_agent.name}' criado usando modelo '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Não foi possível criar agente de Saudação. Verifique chave de API ({greeting_agent.model}). Erro: {e}")

# --- Agente de Despedida ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # Pode usar o mesmo modelo ou um diferente
        model=MODEL_GEMINI_2_0_FLASH,
        # model=LiteLlm(model=MODEL_GPT_4O),  # Se quiser experimentar com outros modelos
        name="farewell_agent",
        instruction="Você é o Agente de Despedida. Sua ÚNICA tarefa é fornecer uma mensagem de despedida educada. "
                    "Use a ferramenta 'say_goodbye' quando o usuário indicar que está saindo ou encerrando a conversa "
                    "(ex: usando palavras como 'tchau', 'adeus', 'obrigado tchau', 'até logo'). "
                    "Não execute outras ações.",
        description="Lida com despedidas simples usando a ferramenta 'say_goodbye'.",  # Crucial para delegação
        tools=[say_goodbye],
    )
    print(f"✅ Agente '{farewell_agent.name}' criado usando modelo '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Não foi possível criar agente de Despedida. Verifique chave de API ({farewell_agent.model}). Erro: {e}")

### 🎯 Delegação Automática Inteligente

O ADK permite **delegação automática**, onde:
- A **descrição** de cada agente é usada pelo modelo orquestrador para decidir qual agente usar
- O agente principal analisa a consulta do usuário e automaticamente delega para o especialista apropriado
- Não precisamos programar regras manuais - o LLM decide baseado no contexto

Isso cria um sistema inteligente de roteamento de tarefas.

In [ ]:
# @title Criando o Agente Raiz com Sub-Agentes
"""
Esta célula cria o agente principal que coordena toda a equipe:
- Mantém a funcionalidade de clima (ferramenta get_weather)
- Adiciona capacidade de delegação para sub-agentes especializados
- Instruções claras sobre quando delegar vs. quando lidar diretamente
"""

# Garante que sub-agentes foram criados com sucesso antes de definir o agente raiz
# Também garante que a ferramenta original 'get_weather' está definida
root_agent = None
runner_root = None  # Inicializa runner

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # Vamos usar um modelo Gemini capaz para o agente raiz lidar com orquestração
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2",  # Novo nome de versão
        model=root_agent_model,
        description="O agente coordenador principal. Lida com solicitações meteorológicas e delega saudações/despedidas para especialistas.",
        instruction="Você é o Agente Meteorológico principal coordenando uma equipe. Sua responsabilidade primária é fornecer informações meteorológicas. "
                    "Use a ferramenta 'get_weather' APENAS para solicitações meteorológicas específicas (ex: 'clima em Londres'). "
                    "Você tem sub-agentes especializados: "
                    "1. 'greeting_agent': Lida com saudações simples como 'Oi', 'Olá'. Delegue para ele nestes casos. "
                    "2. 'farewell_agent': Lida com despedidas simples como 'Tchau', 'Até logo'. Delegue para ele nestes casos. "
                    "Analise a consulta do usuário. Se for saudação, delegue para 'greeting_agent'. Se for despedida, delegue para 'farewell_agent'. "
                    "Se for solicitação meteorológica, lide você mesmo usando 'get_weather'. "
                    "Para qualquer outra coisa, responda adequadamente ou declare que não pode lidar.",
        tools=[get_weather],  # Agente raiz ainda precisa da ferramenta de clima para sua tarefa principal
        # Mudança-chave: Vincular os sub-agentes aqui!
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Agente Raiz '{weather_agent_team.name}' criado usando modelo '{root_agent_model}' com sub-agentes: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Não é possível criar agente raiz porque um ou mais sub-agentes falharam na inicialização ou ferramenta 'get_weather' está ausente.")
    if not greeting_agent: print(" - Agente de Saudação está ausente.")
    if not farewell_agent: print(" - Agente de Despedida está ausente.")
    if 'get_weather' not in globals(): print(" - Função get_weather está ausente.")

In [ ]:
# @title Testando a Equipe de Agentes
"""
Esta célula testa a delegação automática com diferentes tipos de consultas:
1. Saudação -> deve delegar para greeting_agent
2. Clima -> deve usar ferramenta própria get_weather  
3. Despedida -> deve delegar para farewell_agent

Demonstra como o agente principal inteligentemente roteia diferentes solicitações.
"""

import asyncio  # Garante que asyncio está importado

# Garante que o agente raiz (ex: 'weather_agent_team' ou 'root_agent' da célula anterior) está definido
# Garante que a função call_agent_async está definida

# Verifica se a variável do agente raiz existe antes de definir a função de conversa
root_agent_var_name = 'root_agent'  # Nome padrão do guia da Etapa 3
if 'weather_agent_team' in globals():  # Verifica se o usuário usou este nome em vez disso
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Agente raiz ('root_agent' ou 'weather_agent_team') não encontrado. Não é possível definir run_team_conversation.")
    # Atribui um valor fictício para prevenir NameError mais tarde se o bloco de código executar mesmo assim
    root_agent = None  # Ou define uma flag para prevenir execução

# Define e executa apenas se o agente raiz existe
if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    # Define a função async principal para a lógica de conversa
    # As palavras-chave 'await' DENTRO desta função são necessárias para operações async
    async def run_team_conversation():
        print("\n🎭 Testando Delegação da Equipe de Agentes")
        
        # Cria nova sessão para este teste
        session_service = InMemorySessionService()
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"📝 Sessão criada: App='{APP_NAME}', Usuário='{USER_ID}', Sessão='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner(
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"🚀 Runner criado para agente '{actual_root_agent.name}'.")

        # --- Interações usando await (correto dentro de async def) ---
        print("\n🧪 Teste 1: Saudação (deve delegar para greeting_agent)")
        await call_agent_async(query="Olá!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        
        print("\n🧪 Teste 2: Clima (deve usar ferramenta própria)")
        await call_agent_async(query="Qual é o clima em Nova York?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        
        print("\n🧪 Teste 3: Despedida (deve delegar para farewell_agent)")
        await call_agent_async(query="Obrigado, tchau!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # --- Executar a função `run_team_conversation` async ---
    print("🎬 Tentando execução usando 'await' (padrão para notebooks)...")
    await run_team_conversation()

else:
    # Esta mensagem é impressa se a variável do agente raiz não foi encontrada anteriormente
    print("\n⚠️ Pulando execução da conversa da equipe de agentes pois o agente raiz não foi definido com sucesso em etapa anterior.")

--- Tool: say_goodbye called ---
<<< Agent Response: Goodbye! Have a great day.

<<< Agent Response: Goodbye! Have a great day.



## Etapa 4: Adicionando Memória e Personalização com Estado de Sessão

Até agora nossos agentes tinham capacidade de delegação, porém **sem memória** - cada interação começava do zero.

Nesta etapa vamos adicionar:
- **Estado persistente**: Agentes "lembram" de preferências do usuário
- **Memória entre turnos**: Informações importantes são mantidas na sessão
- **output_key**: Respostas do agente são automaticamente salvas no estado
- **Ferramentas state-aware**: Ferramentas que acessam e modificam o estado da sessão

Isso transforma nosso agente em um assistente verdadeiramente personalizado.

In [ ]:
# @title 1. Inicializando Novo Serviço de Sessão com Estado
"""
Esta célula cria uma nova configuração de sessão para demonstrar recursos de estado:
- Novo SessionService dedicado para testes de estado
- Estado inicial com preferências do usuário (Celsius como padrão)
- Novos IDs de sessão para isolar este teste
"""

# Importar componentes de sessão necessários
from google.adk.sessions import InMemorySessionService

# Criar uma NOVA instância do serviço de sessão para esta demonstração de estado
session_service_stateful = InMemorySessionService()
print("✅ Novo InMemorySessionService criado para demonstração de estado.")

# Definir um NOVO ID de sessão para esta parte do tutorial
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Definir dados de estado inicial - usuário prefere Celsius inicialmente
initial_state = {
    "user_preference_temperature_unit": "Celsius"  # Preferência inicial do usuário
}

# Criar a sessão, fornecendo o estado inicial
session_stateful = await session_service_stateful.create_session(
    app_name=APP_NAME,  # Usar o nome de app consistente
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state  # <<< Inicializar estado durante criação
)
print(f"✅ Sessão '{SESSION_ID_STATEFUL}' criada para usuário '{USER_ID_STATEFUL}'.")

# Verificar se o estado inicial foi definido corretamente
retrieved_session = await session_service_stateful.get_session(
    app_name=APP_NAME,
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL
)
print("\n📊 Estado Inicial da Sessão:")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("❌ Erro: Não foi possível recuperar sessão.")

✅ New InMemorySessionService created for state demonstration.
✅ Session 'session_state_demo_001' created for user 'user_state_demo'.

--- Initial Session State ---
{'user_preference_temperature_unit': 'Celsius'}


In [ ]:
# @title 2. Criando Ferramenta Ciente de Estado (State-Aware)
"""
Esta célula cria uma versão avançada da ferramenta get_weather que:
- Acessa preferências do usuário armazenadas no estado da sessão
- Converte temperaturas baseado na preferência (Celsius/Fahrenheit)  
- Atualiza o estado com informações da última cidade consultada
- Demonstra leitura E escrita no estado da sessão
"""

from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """Recupera clima e converte unidade de temperatura baseada no estado da sessão."""
    print(f"🌤️ Ferramenta: get_weather_stateful chamada para {city}")

    # --- Ler preferência do estado ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius")  # Padrão para Celsius
    print(f"📊 Ferramenta: Lendo estado 'user_preference_temperature_unit': {preferred_unit}")

    city_normalized = city.lower().replace(" ", "")

    # Dados meteorológicos mockados (sempre armazenados em Celsius internamente)
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "ensolarado"},
        "london": {"temp_c": 15, "condition": "nublado"},
        "tokyo": {"temp_c": 18, "condition": "chuva leve"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # Formatar temperatura baseada na preferência de estado
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32  # Calcular Fahrenheit
            temp_unit = "°F"
        else:  # Padrão para Celsius
            temp_value = temp_c
            temp_unit = "°C"

        report = f"O clima em {city.capitalize()} está {condition} com temperatura de {temp_value:.0f}{temp_unit}."
        result = {"status": "success", "report": report}
        print(f"📈 Ferramenta: Relatório gerado em {preferred_unit}. Resultado: {result}")

        # Exemplo de escrita de volta no estado (opcional para esta ferramenta)
        tool_context.state["last_city_checked_stateful"] = city
        print(f"💾 Ferramenta: Estado atualizado 'last_city_checked_stateful': {city}")

        return result
    else:
        # Lidar com cidade não encontrada
        error_msg = f"Desculpe, não tenho informações meteorológicas para '{city}'."
        print(f"❌ Ferramenta: Cidade '{city}' não encontrada.")
        return {"status": "error", "error_message": error_msg}

print("✅ Ferramenta 'get_weather_stateful' ciente de estado definida.")

✅ State-aware 'get_weather_stateful' tool defined.


In [ ]:
# @title 3. Redefinindo Sub-Agentes e Agente Raiz com output_key
"""
Esta célula cria uma nova versão dos agentes com recursos avançados:
- Sub-agentes redefinidos para garantir compatibilidade
- Agente raiz com ferramenta state-aware (get_weather_stateful)
- output_key: Respostas finais são automaticamente salvas no estado
- Runner configurado com o novo serviço de sessão stateful
"""

# Garantir imports necessários: Agent, Runner
from google.adk.agents import Agent
from google.adk.runners import Runner
# Garantir que ferramentas 'say_hello', 'say_goodbye' estão definidas (da Etapa 3)
# Garantir que constantes de modelo MODEL_GEMINI_2_0_FLASH etc. estão definidas

# --- Redefinir Agente de Saudação (da Etapa 3) ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="Você é o Agente de Saudação. Sua ÚNICA tarefa é fornecer uma saudação amigável usando a ferramenta 'say_hello'. Não faça mais nada.",
        description="Lida com saudações simples usando a ferramenta 'say_hello'.",
        tools=[say_hello],
    )
    print(f"✅ Agente '{greeting_agent.name}' redefinido.")
except Exception as e:
    print(f"❌ Não foi possível redefinir agente de Saudação. Erro: {e}")

# --- Redefinir Agente de Despedida (da Etapa 3) ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="Você é o Agente de Despedida. Sua ÚNICA tarefa é fornecer uma mensagem de despedida educada usando a ferramenta 'say_goodbye'. Não execute outras ações.",
        description="Lida com despedidas simples usando a ferramenta 'say_goodbye'.",
        tools=[say_goodbye],
    )
    print(f"✅ Agente '{farewell_agent.name}' redefinido.")
except Exception as e:
    print(f"❌ Não foi possível redefinir agente de Despedida. Erro: {e}")

# --- Definir o Agente Raiz Atualizado ---
root_agent_stateful = None
runner_root_stateful = None  # Inicializar runner

# Verificar pré-requisitos antes de criar o agente raiz
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH  # Escolher modelo de orquestração

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful",  # Novo nome de versão
        model=root_agent_model,
        description="Agente principal: Fornece clima (unidade ciente de estado), delega saudações/despedidas, salva relatório no estado.",
        instruction="Você é o Agente Meteorológico principal. Seu trabalho é fornecer clima usando 'get_weather_stateful'. "
                    "A ferramenta formatará a temperatura baseada na preferência do usuário armazenada no estado. "
                    "Delegue saudações simples para 'greeting_agent' e despedidas para 'farewell_agent'. "
                    "Lide apenas com solicitações meteorológicas, saudações e despedidas.",
        tools=[get_weather_stateful],  # Usar a ferramenta ciente de estado
        sub_agents=[greeting_agent, farewell_agent],  # Incluir sub-agentes
        output_key="last_weather_report"  # <<< Auto-salvar resposta meteorológica final do agente
    )
    print(f"✅ Agente Raiz '{root_agent_stateful.name}' criado usando ferramenta stateful e output_key.")

    # --- Criar Runner para este Agente Raiz & NOVO Serviço de Sessão ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful  # Usar o NOVO serviço de sessão stateful
    )
    print(f"✅ Runner criado para agente raiz stateful '{runner_root_stateful.agent.name}' usando serviço de sessão stateful.")

else:
    print("❌ Não é possível criar agente raiz stateful. Pré-requisitos ausentes.")
    if not greeting_agent: print(" - definição de greeting_agent ausente.")
    if not farewell_agent: print(" - definição de farewell_agent ausente.")
    if 'get_weather_stateful' not in globals(): print(" - ferramenta get_weather_stateful ausente.")

✅ Agent 'greeting_agent' redefined.
✅ Agent 'farewell_agent' redefined.
✅ Root Agent 'weather_agent_v4_stateful' created using stateful tool and output_key.
✅ Runner created for stateful root agent 'weather_agent_v4_stateful' using stateful session service.


In [ ]:
# @title Função de Interação com Controle de Rate Limiting
"""
Esta célula cria uma versão aprimorada da função de interação que inclui:
- Controle de rate limiting para evitar erros de quota da API
- Retry automático quando limite de taxa é excedido
- Delay configurável entre chamadas
- Tratamento robusto de erros 429 (RESOURCE_EXHAUSTED)
"""

from google.genai import types  # Para criar conteúdo/partes de mensagens
import asyncio

async def call_agent_async(query: str, runner, user_id, session_id, delay=5):
    """Envia consulta para agente e exibe resposta final com controle de rate limiting."""
    print(f"\n💭 Consulta do Usuário: {query}")
    
    # Adicionar delay entre requisições para respeitar limites de taxa
    await asyncio.sleep(delay)

    # Preparar mensagem do usuário no formato ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "Agente não produziu uma resposta final."  # Padrão

    try:
        # Conceito-chave: run_async executa lógica do agente e gera Eventos
        # Iteramos através dos eventos para encontrar a resposta final
        async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
            # Descomente a linha abaixo para ver *todos* os eventos durante execução
            # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Conteúdo: {event.content}")

            # Conceito-chave: is_final_response() marca a mensagem conclusiva do turno
            if event.is_final_response():
                if event.content and event.content.parts:
                   # Assumindo resposta de texto na primeira parte
                   final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate:  # Lidar com possíveis erros/escalações
                   final_response_text = f"Agente escalou: {event.error_message or 'Nenhuma mensagem específica.'}"
                # Adicionar mais verificações aqui se necessário (ex: códigos de erro específicos)
                break  # Parar processamento de eventos após encontrar resposta final
                
    except Exception as e:
        # Tratamento específico para erros de rate limiting
        if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
            print("⚠️ Limite de taxa excedido. Aguardando 60 segundos e tentando novamente...")
            await asyncio.sleep(60)
            return await call_agent_async(query, runner, user_id, session_id, delay)
        else:
            final_response_text = f"Erro ocorrido: {str(e)}"

    print(f"🤖 Resposta do Agente: {final_response_text}")

In [ ]:
# @title 4. Testando Fluxo de Estado e output_key com Rate Limiting
"""
Esta célula executa um teste abrangente dos recursos de estado:
1. Primeira consulta: Usa preferência inicial (Celsius)
2. Modificação manual do estado: Muda para Fahrenheit
3. Segunda consulta: Verifica se nova preferência é aplicada
4. Teste de delegação: Verifica se sub-agentes ainda funcionam
5. Inspeção final: Examina estado completo da sessão

Inclui delays aumentados (8s) para evitar problemas de quota.
"""

import asyncio  # Garantir que asyncio está importado

# Garantir que runner stateful está disponível da célula anterior
if 'runner_root_stateful' in globals() and runner_root_stateful:
    # Definir função async principal para lógica de conversa stateful
    async def run_stateful_conversation():
        print("\n🧪 Testando Estado: Conversão de Unidade de Temperatura & output_key (com rate limiting)")

        # 1. Verificar clima (Usa estado inicial: Celsius)
        print("\n--- Turno 1: Solicitando clima em Londres (esperar Celsius) ---")
        await call_agent_async(query="Qual é o clima em Londres?",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL,
                               delay=8  # Delay aumentado para evitar rate limiting
                              )

        # 2. Atualizar manualmente preferência de estado para Fahrenheit - MODIFICAÇÃO DIRETA DO ARMAZENAMENTO
        print("\n--- Atualizando Estado Manualmente: Definindo unidade para Fahrenheit ---")
        try:
            # Acessar armazenamento interno diretamente - ESPECÍFICO para InMemorySessionService para testes
            # NOTA: Em produção com serviços persistentes (Database, VertexAI), você normalmente
            # atualizaria estado via ações do agente ou APIs de serviço específicas se disponível,
            # não por manipulação direta do armazenamento interno.
            stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
            stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
            print(f"📊 Estado da sessão armazenada atualizado. 'user_preference_temperature_unit' atual: {stored_session.state.get('user_preference_temperature_unit', 'Não Definido')}")
        except KeyError:
            print(f"❌ Erro: Não foi possível recuperar sessão '{SESSION_ID_STATEFUL}' do armazenamento interno para usuário '{USER_ID_STATEFUL}' no app '{APP_NAME}' para atualizar estado. Verifique IDs e se sessão foi criada.")
        except Exception as e:
             print(f"❌ Erro atualizando estado interno da sessão: {e}")

        # 3. Verificar clima novamente (Ferramenta deve agora usar Fahrenheit)
        print("\n--- Turno 2: Solicitando clima em Nova York (esperar Fahrenheit) ---")
        await call_agent_async(query="Me fale sobre o clima em Nova York.",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL,
                               delay=8  # Delay aumentado
                              )

        # 4. Testar delegação básica (deve ainda funcionar)
        print("\n--- Turno 3: Enviando saudação ---")
        await call_agent_async(query="Oi!",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL,
                               delay=8  # Delay aumentado
                              )

    # --- Executar função `run_stateful_conversation` ---
    print("🎬 Executando com rate limiting (8 segundos entre chamadas)...")
    await run_stateful_conversation()

    # --- Inspecionar estado final da sessão após conversa ---
    print("\n📊 Inspecionando Estado Final da Sessão")
    final_session = await session_service_stateful.get_session(
        app_name=APP_NAME,
        user_id=USER_ID_STATEFUL,
        session_id=SESSION_ID_STATEFUL
    )
    if final_session:
        # Usar .get() para acesso mais seguro a chaves potencialmente ausentes
        print(f"Preferência Final: {final_session.state.get('user_preference_temperature_unit', 'Não Definido')}")
        print(f"Último Relatório Meteorológico (do output_key): {final_session.state.get('last_weather_report', 'Não Definido')}")
        print(f"Última Cidade Verificada (pela ferramenta): {final_session.state.get('last_city_checked_stateful', 'Não Definido')}")
    else:
        print("❌ Erro: Não foi possível recuperar estado final da sessão.")

else:
    print("\n⚠️ Pulando teste de conversa de estado. Runner do agente raiz stateful ('runner_root_stateful') não está disponível.")

Executing with rate limiting (8 seconds between calls)...

--- Testing State: Temp Unit Conversion & output_key (with rate limiting) ---
--- Turn 1: Requesting weather in London (expect Celsius) ---

>>> User Query: What's the weather in London?
--- Tool: get_weather_stateful called for London ---
--- Tool: Reading state 'user_preference_temperature_unit': Celsius ---
--- Tool: Generated report in Celsius. Result: {'status': 'success', 'report': 'The weather in London is cloudy with a temperature of 15°C.'} ---
--- Tool: Updated state 'last_city_checked_stateful': London ---
--- Tool: get_weather_stateful called for London ---
--- Tool: Reading state 'user_preference_temperature_unit': Celsius ---
--- Tool: Generated report in Celsius. Result: {'status': 'success', 'report': 'The weather in London is cloudy with a temperature of 15°C.'} ---
--- Tool: Updated state 'last_city_checked_stateful': London ---
⚠️ Rate limit exceeded. Waiting 60 seconds and retrying...
⚠️ Rate limit exceeded.

## Etapa 5: Adicionando Guardrails de Segurança - Proteção contra Alucinações

**Guardrails** são mecanismos de segurança que protegem nosso agente contra:
- Comportamentos indesejados 
- Respostas inadequadas
- Uso incorreto de ferramentas
- Violações de políticas de uso

Nesta etapa implementamos **guardrails de entrada de modelo** (`before_model_callback`) que:
- Interceptam mensagens antes de chegarem ao LLM
- Bloqueiam palavras-chave ou conteúdo indesejado  
- Retornam respostas predefinidas quando necessário
- Registram eventos de segurança no estado da sessão

Isso torna o modelo mais confiável e adequado para uso em produção.

In [ ]:
# @title 1. Definindo Guardrail before_model_callback
"""
Esta célula cria um guardrail que intercepta mensagens ANTES de serem enviadas ao modelo LLM.
Funcionalidades:
- Verifica se a mensagem contém palavras-chave bloqueadas (ex: "BLOCK")
- Se encontrar, bloqueia a chamada ao LLM e retorna resposta predefinida
- Registra o evento no estado da sessão para auditoria
- Permite que mensagens normais passem normalmente
"""

# Garantir que imports necessários estão disponíveis
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types  # Para criar conteúdo de resposta
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Inspeciona a última mensagem do usuário em busca de 'BLOCK'. Se encontrar, bloqueia 
    a chamada LLM e retorna uma LlmResponse predefinida. Caso contrário, retorna None para prosseguir.
    """
    agent_name = callback_context.agent_name  # Nome do agente cuja chamada de modelo está sendo interceptada
    print(f"🛡️ Callback: block_keyword_guardrail executando para agente: {agent_name}")

    # Extrair texto da última mensagem do usuário no histórico da requisição
    last_user_message_text = ""
    if llm_request.contents:
        # Encontrar a mensagem mais recente com role 'user'
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # Assumindo que texto está na primeira parte por simplicidade
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break  # Encontrou o texto da última mensagem do usuário

    print(f"🔍 Callback: Inspecionando última mensagem do usuário: '{last_user_message_text[:100]}...'")

    # --- Lógica do Guardrail ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper():  # Verificação case-insensitive
        print(f"🚫 Callback: Encontrou '{keyword_to_block}'. Bloqueando chamada LLM!")
        
        # Opcionalmente, definir flag no estado para registrar evento de bloqueio
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"📝 Callback: Definiu estado 'guardrail_block_keyword_triggered': True")

        # Construir e retornar LlmResponse para parar o fluxo e enviar esta resposta em vez disso
        return LlmResponse(
            content=types.Content(
                role="model",  # Simular resposta da perspectiva do agente
                parts=[types.Part(text=f"Não posso processar esta solicitação porque contém a palavra-chave bloqueada '{keyword_to_block}'.")],
            )
            # Nota: Você também pode definir um campo error_message aqui se necessário
        )
    else:
        # Palavra-chave não encontrada, permitir que requisição prossiga para o LLM
        print(f"✅ Callback: Palavra-chave não encontrada. Permitindo chamada LLM para {agent_name}.")
        return None  # Retornar None sinaliza ao ADK para continuar normalmente

print("✅ Função block_keyword_guardrail definida.")

✅ block_keyword_guardrail function defined.


In [58]:
# @title 2. Update Root Agent with before_model_callback


# --- Redefine Sub-Agents (Ensures they exist in this context) ---
greeting_agent = None
try:
    # Use a defined model constant
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({greeting_agent.model}). Error: {e}")

farewell_agent = None
try:
    # Use a defined model constant
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({farewell_agent.model}). Error: {e}")


# --- Define the Root Agent with the Callback ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# Check all components before proceeding
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

    # Use a defined model constant
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # New version name for clarity
        model=root_agent_model,
        description="Main agent: Handles weather, delegates greetings/farewells, includes input keyword guardrail.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent], # Reference the redefined sub-agents
        output_key="last_weather_report", # Keep output_key from Step 4
        before_model_callback=block_keyword_guardrail # <<< Assign the guardrail callback
    )
    print(f"✅ Root Agent '{root_agent_model_guardrail.name}' created with before_model_callback.")

    # --- Create Runner for this Agent, Using SAME Stateful Session Service ---
    # Ensure session_service_stateful exists from Step 4
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # Use consistent APP_NAME
            session_service=session_service_stateful # <<< Use the service from Step 4
        )
        print(f"✅ Runner created for guardrail agent '{runner_root_model_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4 is missing.")

else:
    print("❌ Cannot create root agent with model guardrail. One or more prerequisites are missing or failed initialization:")
    if not greeting_agent: print("   - Greeting Agent")
    if not farewell_agent: print("   - Farewell Agent")
    if 'get_weather_stateful' not in globals(): print("   - 'get_weather_stateful' tool")
    if 'block_keyword_guardrail' not in globals(): print("   - 'block_keyword_guardrail' callback")

✅ Sub-Agent 'greeting_agent' redefined.
✅ Sub-Agent 'farewell_agent' redefined.
✅ Root Agent 'weather_agent_v5_model_guardrail' created with before_model_callback.
✅ Runner created for guardrail agent 'weather_agent_v5_model_guardrail', using stateful session service.


In [59]:
# @title 3. Interact to Test the Model Input Guardrail
import asyncio # Ensure asyncio is imported

# Ensure the runner for the guardrail agent is available
if 'runner_root_model_guardrail' in globals() and runner_root_model_guardrail:
    # Define the main async function for the guardrail test conversation.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_guardrail_test_conversation():
        print("\n--- Testing Model Input Guardrail ---")

        # Use the runner for the agent with the callback and the existing stateful session ID
        # Define a helper lambda for cleaner interaction calls
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_model_guardrail,
                                                         USER_ID_STATEFUL, # Use existing user ID
                                                         SESSION_ID_STATEFUL # Use existing session ID
                                                        )
        # 1. Normal request (Callback allows, should use Fahrenheit from previous state change)
        print("--- Turn 1: Requesting weather in London (expect allowed, Fahrenheit) ---")
        await interaction_func("What is the weather in London?")

        # 2. Request containing the blocked keyword (Callback intercepts)
        print("\n--- Turn 2: Requesting with blocked keyword (expect blocked) ---")
        await interaction_func("BLOCK the request for weather in Tokyo") # Callback should catch "BLOCK"

        # 3. Normal greeting (Callback allows root agent, delegation happens)
        print("\n--- Turn 3: Sending a greeting (expect allowed) ---")
        await interaction_func("Hello again")

    # --- Execute the `run_guardrail_test_conversation` async function ---
    # Choose ONE of the methods below based on your environment.

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_guardrail_test_conversation()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_guardrail_test_conversation()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_guardrail_test_conversation())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

    # --- Inspect final session state after the conversation ---
    # This block runs after either execution method completes.
    # Optional: Check state for the trigger flag set by the callback
    print("\n--- Inspecting Final Session State (After Guardrail Test) ---")
    # Use the session service instance associated with this stateful session
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        # Use .get() for safer access
        print(f"Guardrail Triggered Flag: {final_session.state.get('guardrail_block_keyword_triggered', 'Not Set (or False)')}")
        print(f"Last Weather Report: {final_session.state.get('last_weather_report', 'Not Set')}") # Should be London weather if successful
        print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}") # Should be Fahrenheit
        # print(f"Full State Dict: {final_session.state}") # For detailed view
    else:
        print("\n❌ Error: Could not retrieve final session state.")

else:
    print("\n⚠️ Skipping model guardrail test. Runner ('runner_root_model_guardrail') is not available.")

Attempting execution using 'await' (default for notebooks)...

--- Testing Model Input Guardrail ---
--- Turn 1: Requesting weather in London (expect allowed, Fahrenheit) ---

>>> User Query: What is the weather in London?
--- Callback: block_keyword_guardrail running for agent: weather_agent_v5_model_guardrail ---
--- Callback: Inspecting last user message: 'For context:...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v5_model_guardrail. ---
--- Callback: block_keyword_guardrail running for agent: weather_agent_v5_model_guardrail ---
--- Callback: Inspecting last user message: 'For context:...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v5_model_guardrail. ---
--- Tool: get_weather_stateful called for London ---
--- Tool: Reading state 'user_preference_temperature_unit': Fahrenheit ---
--- Tool: Generated report in Fahrenheit. Result: {'status': 'success', 'report': 'The weather in London is cloudy with a temperature of 59°F.'

## Etapa 6: Guardrails de Segurança para Decisões de Ferramentas

`before_tool_callback` é a forma de **proteger e validar argumentos de ferramentas**, permitindo controle fino sobre:

- **Quais ferramentas** podem ser executadas
- **Com quais argumentos** podem ser chamadas  
- **Em quais condições** são permitidas
- **Políticas específicas** da aplicação

Este tipo de guardrail é essencial para:
- Prevenir uso inadequado de ferramentas
- Implementar políticas de negócio
- Proteger recursos sensíveis
- Auditoria e compliance

Exemplo: Bloquear consultas meteorológicas para certas cidades por motivos de política.

In [ ]:
# @title 1. Definindo Guardrail before_tool_callback
"""
Esta célula cria um guardrail que intercepta chamadas de ferramentas ANTES da execução.
Funcionalidades:
- Verifica se get_weather_stateful está sendo chamada para 'Paris'
- Se sim, bloqueia a execução e retorna erro específico
- Registra evento de bloqueio no estado da sessão
- Permite outras cidades e ferramentas normalmente
"""

# Garantir que imports necessários estão disponíveis
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any  # Para type hints

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Verifica se 'get_weather_stateful' está sendo chamada para 'Paris'.
    Se sim, bloqueia a execução da ferramenta e retorna dicionário de erro específico.
    Caso contrário, permite que chamada da ferramenta prossiga retornando None.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name  # Agente tentando fazer a chamada da ferramenta
    print(f"🛡️ Callback: block_paris_tool_guardrail executando para ferramenta '{tool_name}' no agente '{agent_name}'")
    print(f"🔍 Callback: Inspecionando args: {args}")

    # --- Lógica do Guardrail ---
    target_tool_name = "get_weather_stateful"  # Corresponder ao nome da função usado por FunctionTool
    blocked_city = "paris"

    # Verificar se é a ferramenta correta e o argumento cidade corresponde à cidade bloqueada
    if tool_name == target_tool_name:
        city_argument = args.get("city", "")  # Obter com segurança o argumento 'city'
        if city_argument and city_argument.lower() == blocked_city:
            print(f"🚫 Callback: Detectada cidade bloqueada '{city_argument}'. Bloqueando execução da ferramenta!")
            
            # Opcionalmente atualizar estado
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"📝 Callback: Definiu estado 'guardrail_tool_block_triggered': True")

            # Retornar dicionário correspondendo ao formato de saída esperado da ferramenta para erros
            # Este dicionário se torna o resultado da ferramenta, pulando a execução real da ferramenta
            return {
                "status": "error",
                "error_message": f"Restrição de política: Verificações de clima para '{city_argument.capitalize()}' estão atualmente desabilitadas por um guardrail de ferramenta."
            }
        else:
             print(f"✅ Callback: Cidade '{city_argument}' é permitida para ferramenta '{tool_name}'.")
    else:
        print(f"✅ Callback: Ferramenta '{tool_name}' não é a ferramenta alvo. Permitindo.")

    # Se as verificações acima não retornaram um dicionário, permitir que ferramenta execute
    print(f"✅ Callback: Permitindo ferramenta '{tool_name}' prosseguir.")
    return None  # Retornar None permite que função real da ferramenta execute

print("✅ Função block_paris_tool_guardrail definida.")

✅ block_paris_tool_guardrail function defined.


In [61]:
# @title 2. Update Root Agent with BOTH Callbacks (Self-Contained)

# --- Ensure Prerequisites are Defined ---
# (Include or ensure execution of definitions for: Agent, LiteLlm, Runner, ToolContext,
#  MODEL constants, say_hello, say_goodbye, greeting_agent, farewell_agent,
#  get_weather_stateful, block_keyword_guardrail, block_paris_tool_guardrail)

# --- Redefine Sub-Agents (Ensures they exist in this context) ---
greeting_agent = None
try:
    # Use a defined model constant
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({greeting_agent.model}). Error: {e}")

farewell_agent = None
try:
    # Use a defined model constant
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({farewell_agent.model}). Error: {e}")

# --- Define the Root Agent with Both Callbacks ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # New version name
        model=root_agent_model,
        description="Main agent: Handles weather, delegates, includes input AND tool guardrails.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # Keep model guardrail
        before_tool_callback=block_paris_tool_guardrail # <<< Add tool guardrail
    )
    print(f"✅ Root Agent '{root_agent_tool_guardrail.name}' created with BOTH callbacks.")

    # --- Create Runner, Using SAME Stateful Session Service ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< Use the service from Step 4/5
        )
        print(f"✅ Runner created for tool guardrail agent '{runner_root_tool_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4/5 is missing.")

else:
    print("❌ Cannot create root agent with tool guardrail. Prerequisites missing.")

✅ Sub-Agent 'greeting_agent' redefined.
✅ Sub-Agent 'farewell_agent' redefined.
✅ Root Agent 'weather_agent_v6_tool_guardrail' created with BOTH callbacks.
✅ Runner created for tool guardrail agent 'weather_agent_v6_tool_guardrail', using stateful session service.


In [62]:
# @title 3. Interact to Test the Tool Argument Guardrail
import asyncio # Ensure asyncio is imported

# Ensure the runner for the tool guardrail agent is available
if 'runner_root_tool_guardrail' in globals() and runner_root_tool_guardrail:
    # Define the main async function for the tool guardrail test conversation.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_tool_guardrail_test():
        print("\n--- Testing Tool Argument Guardrail ('Paris' blocked) ---")

        # Use the runner for the agent with both callbacks and the existing stateful session
        # Define a helper lambda for cleaner interaction calls
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_tool_guardrail,
                                                         USER_ID_STATEFUL, # Use existing user ID
                                                         SESSION_ID_STATEFUL # Use existing session ID
                                                        )
        # 1. Allowed city (Should pass both callbacks, use Fahrenheit state)
        print("--- Turn 1: Requesting weather in New York (expect allowed) ---")
        await interaction_func("What's the weather in New York?")

        # 2. Blocked city (Should pass model callback, but be blocked by tool callback)
        print("\n--- Turn 2: Requesting weather in Paris (expect blocked by tool guardrail) ---")
        await interaction_func("How about Paris?") # Tool callback should intercept this

        # 3. Another allowed city (Should work normally again)
        print("\n--- Turn 3: Requesting weather in London (expect allowed) ---")
        await interaction_func("Tell me the weather in London.")

    # --- Execute the `run_tool_guardrail_test` async function ---
    # Choose ONE of the methods below based on your environment.

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_tool_guardrail_test()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_tool_guardrail_test()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_tool_guardrail_test())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

    # --- Inspect final session state after the conversation ---
    # This block runs after either execution method completes.
    # Optional: Check state for the tool block trigger flag
    print("\n--- Inspecting Final Session State (After Tool Guardrail Test) ---")
    # Use the session service instance associated with this stateful session
    final_session = await session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id= SESSION_ID_STATEFUL)
    if final_session:
        # Use .get() for safer access
        print(f"Tool Guardrail Triggered Flag: {final_session.state.get('guardrail_tool_block_triggered', 'Not Set (or False)')}")
        print(f"Last Weather Report: {final_session.state.get('last_weather_report', 'Not Set')}") # Should be London weather if successful
        print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}") # Should be Fahrenheit
        # print(f"Full State Dict: {final_session.state}") # For detailed view
    else:
        print("\n❌ Error: Could not retrieve final session state.")

else:
    print("\n⚠️ Skipping tool guardrail test. Runner ('runner_root_tool_guardrail') is not available.")

Attempting execution using 'await' (default for notebooks)...

--- Testing Tool Argument Guardrail ('Paris' blocked) ---
--- Turn 1: Requesting weather in New York (expect allowed) ---

>>> User Query: What's the weather in New York?
<<< Agent Response: Error occurred: Agent weather_agent_v5_model_guardrail not found in the agent tree.

--- Turn 2: Requesting weather in Paris (expect blocked by tool guardrail) ---

>>> User Query: How about Paris?
<<< Agent Response: Error occurred: Agent weather_agent_v5_model_guardrail not found in the agent tree.

--- Turn 2: Requesting weather in Paris (expect blocked by tool guardrail) ---

>>> User Query: How about Paris?
--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_guardrail ---
--- Callback: Inspecting last user message: 'For context:...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v6_tool_guardrail. ---
--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_